In [2]:
# imports
import pandas as pd
import numpy as np
import feature_engine 
import pickle

In [126]:
# Constants:
RAW_DATA_PATH = '../data/raw/survey_results_public.csv'
PROCESSED_DATA_PATH = '../data/processed'
TARGET_COL = 'DevType'
with open('../data/variables/possible_predictors.pkl', 'rb') as f:
    PREDICTOR_COLS = pickle.load(f)

### Helper functions:

In [116]:
# functions:
def check_multiple_answers(df, col):
    return df[col].str.contains(';').any() == True

def split_answers(df, col):
    df_col = df[col].replace(np.nan, '') 
    return df_col.str.split(';')      

In [117]:
df_raw = pd.read_csv(RAW_DATA_PATH).loc[:, PREDICTOR_COLS ]
df_raw.sample(3, random_state=42).T


,76172,30324,25768
EdLevel,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)","Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Some college/university study without earning ...
YearsCode,5,11,4
YearsCodePro,7,9,Less than 1 year
DevType,"Developer, front-end;Developer, desktop or ent...","Developer, mobile;Academic researcher","Developer, desktop or enterprise applications;..."
OrgSize,100 to 499 employees,20 to 99 employees,20 to 99 employees
LanguageHaveWorkedWith,C;C#;PHP,Objective-C;Python;Swift,C#;HTML/CSS;SQL
LanguageWantToWorkWith,C;C#;HTML/CSS;JavaScript;TypeScript,Objective-C;Python;Swift,Bash/Shell;C#;F#;JavaScript;PowerShell;Python;SQL
DatabaseHaveWorkedWith,Firebase,SQLite,Microsoft SQL Server;MySQL
DatabaseWantToWorkWith,DynamoDB;Elasticsearch;Firebase;MongoDB;MySQL;...,SQLite,MariaDB;Microsoft SQL Server;Oracle
PlatformHaveWorkedWith,AWS;DigitalOcean,NaN,NaN


# Preprocessing:

## Data Type conversion
- Data type conversion: `YearsCode` , `YearsCodePro` from object to int16
    
    Issues: 'Less than 1 year' ,'More than 50 years' , nan

In [118]:
df_processed = df_raw.copy()
# replacing 'Less than 1 year' with 0

df_processed.loc[df_processed['YearsCode'] == 'Less than 1 year', 'YearsCode'] = '0'
df_processed.loc[df_processed['YearsCodePro'] == 'Less than 1 year', 'YearsCodePro'] = '0'

# replacing 'More than 50 years' with 51
df_processed.loc[df_processed['YearsCode'] == 'More than 50 years', 'YearsCode'] = '51'
df_processed.loc[df_processed['YearsCodePro'] == 'More than 50 years', 'YearsCodePro'] = '51'


# converting to numeric
df_processed['YearsCode'] = df_processed['YearsCode'].astype(np.float32)
df_processed['YearsCodePro'] = df_processed['YearsCodePro'].astype(np.float32)

In [119]:
df_processed.dtypes

EdLevel                          object
YearsCode                       float32
YearsCodePro                    float32
DevType                          object
OrgSize                          object
LanguageHaveWorkedWith           object
LanguageWantToWorkWith           object
DatabaseHaveWorkedWith           object
DatabaseWantToWorkWith           object
PlatformHaveWorkedWith           object
PlatformWantToWorkWith           object
WebframeHaveWorkedWith           object
WebframeWantToWorkWith           object
MiscTechHaveWorkedWith           object
MiscTechWantToWorkWith           object
ToolsTechHaveWorkedWith          object
ToolsTechWantToWorkWith          object
NEWCollabToolsHaveWorkedWith     object
NEWCollabToolsWantToWorkWith     object
OpSys                            object
dtype: object

## Needed preprocessing:
-  Split Tools `;`

In [120]:
cat_cols = df_processed.select_dtypes(include=['object']).columns.tolist()
for col in cat_cols:
    print(f'Column {col} :')
    if check_multiple_answers(df_processed, col):
        print('Contains multiple answers')
        df_processed[col] = split_answers(df_processed, col)
        print('Splited')
    else:
        print('No multiple answers')
    print(f'\n','---'*5)
print('Done')


Column EdLevel :
No multiple answers

 ---------------
Column DevType :
Contains multiple answers
Splited

 ---------------
Column OrgSize :
No multiple answers

 ---------------
Column LanguageHaveWorkedWith :
Contains multiple answers
Splited

 ---------------
Column LanguageWantToWorkWith :
Contains multiple answers
Splited

 ---------------
Column DatabaseHaveWorkedWith :
Contains multiple answers
Splited

 ---------------
Column DatabaseWantToWorkWith :
Contains multiple answers
Splited

 ---------------
Column PlatformHaveWorkedWith :
Contains multiple answers
Splited

 ---------------
Column PlatformWantToWorkWith :
Contains multiple answers
Splited

 ---------------
Column WebframeHaveWorkedWith :
Contains multiple answers
Splited

 ---------------
Column WebframeWantToWorkWith :
Contains multiple answers
Splited

 ---------------
Column MiscTechHaveWorkedWith :
Contains multiple answers
Splited

 ---------------
Column MiscTechWantToWorkWith :
Contains multiple answers
Splited

##### Verifying results

In [121]:
index = df_processed.sample(3).index
display(
    df_raw.loc[index, :] , 
    df_processed.loc[index, :]
)

,EdLevel,YearsCode,YearsCodePro,DevType,OrgSize,LanguageHaveWorkedWith,LanguageWantToWorkWith,DatabaseHaveWorkedWith,DatabaseWantToWorkWith,PlatformHaveWorkedWith,PlatformWantToWorkWith,WebframeHaveWorkedWith,WebframeWantToWorkWith,MiscTechHaveWorkedWith,MiscTechWantToWorkWith,ToolsTechHaveWorkedWith,ToolsTechWantToWorkWith,NEWCollabToolsHaveWorkedWith,NEWCollabToolsWantToWorkWith,OpSys
46324,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",12,2,"Developer, back-end;Developer, game or graphic...",10 to 19 employees,Bash/Shell;C;C#;C++;HTML/CSS;Node.js;PowerShel...,C#;C++;PowerShell;Rust,NaN,NaN,NaN,NaN,NaN,NaN,.NET Framework,.NET Framework;.NET Core / .NET 5,Docker;Git;Unity 3D;Unreal Engine;Yarn,Git;Unity 3D;Unreal Engine,Sublime Text;Vim;Visual Studio;Visual Studio Code,Sublime Text;Vim;Visual Studio;Visual Studio Code,Windows
49933,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",6,4,"Developer, full-stack;Developer, back-end","1,000 to 4,999 employees",Elixir;HTML/CSS;Ruby,Clojure;Elixir;Ruby,PostgreSQL,PostgreSQL,AWS,AWS,jQuery;Ruby on Rails,React.js;Ruby on Rails;Vue.js,NaN,NaN,Docker;Terraform,Docker;Terraform,Vim;Visual Studio Code,Vim,Linux-based
18645,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",32,19,"Developer, desktop or enterprise applications;...",2 to 9 employees,C;C++;Delphi;PowerShell,C;Delphi;Julia;PowerShell,SQLite,SQLite,NaN,NaN,NaN,NaN,NaN,NaN,Git,NaN,Notepad++,Notepad++,Windows


,EdLevel,YearsCode,YearsCodePro,DevType,OrgSize,LanguageHaveWorkedWith,LanguageWantToWorkWith,DatabaseHaveWorkedWith,DatabaseWantToWorkWith,PlatformHaveWorkedWith,PlatformWantToWorkWith,WebframeHaveWorkedWith,WebframeWantToWorkWith,MiscTechHaveWorkedWith,MiscTechWantToWorkWith,ToolsTechHaveWorkedWith,ToolsTechWantToWorkWith,NEWCollabToolsHaveWorkedWith,NEWCollabToolsWantToWorkWith,OpSys
46324,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",12.0,2.0,"[Developer, back-end, Developer, game or graph...",10 to 19 employees,"[Bash/Shell, C, C#, C++, HTML/CSS, Node.js, Po...","[C#, C++, PowerShell, Rust]",[],[],[],[],[],[],[.NET Framework],"[.NET Framework, .NET Core / .NET 5]","[Docker, Git, Unity 3D, Unreal Engine, Yarn]","[Git, Unity 3D, Unreal Engine]","[Sublime Text, Vim, Visual Studio, Visual Stud...","[Sublime Text, Vim, Visual Studio, Visual Stud...",Windows
49933,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",6.0,4.0,"[Developer, full-stack, Developer, back-end]","1,000 to 4,999 employees","[Elixir, HTML/CSS, Ruby]","[Clojure, Elixir, Ruby]",[PostgreSQL],[PostgreSQL],[AWS],[AWS],"[jQuery, Ruby on Rails]","[React.js, Ruby on Rails, Vue.js]",[],[],"[Docker, Terraform]","[Docker, Terraform]","[Vim, Visual Studio Code]",[Vim],Linux-based
18645,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",32.0,19.0,"[Developer, desktop or enterprise applications...",2 to 9 employees,"[C, C++, Delphi, PowerShell]","[C, Delphi, Julia, PowerShell]",[SQLite],[SQLite],[],[],[],[],[],[],[Git],[],[Notepad++],[Notepad++],Windows


### Export Data

In [127]:
df_processed.to_pickle(f'{PROCESSED_DATA_PATH}/1_preprocessed_df.pkl')